In [1]:
import numpy as np
import bilby
from gwpy.timeseries import TimeSeries
import matplotlib.pyplot as plt
import lal

import sys
sys.path.append("..")

from create_post_dict import create_post_dict, extract_relevant_info

waveform = 'C01:IMRPhenomXPHM'
file_path = '/home/shunyin.cheung/GWOSC_PE_runs/posteriors/IGWN-GWTC2p1-v2-GW151012_095443_PEDataRelease_mixed_cosmo.h5'

# define parameters
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

waveform_name = 'IMRPhenomXPHM'

calib_file_paths={"H1":"H1.dat", "L1":"L1.dat", }

logger = bilby.core.utils.logger

ifo_list = bilby.gw.detector.InterferometerList([])

# set up ifos
for det in detectors: 
    ifo = bilby.gw.detector.get_empty_interferometer(det)

    logger.info("Downloading analysis data for ifo {}".format(det))
    # call analysis data
    data = TimeSeries.fetch_open_data(det, start_time, end_time, sample_rate=16384)

    # Resampling timeseries to sampling_frequency using lal.
    lal_timeseries = data.to_lal()
    lal.ResampleREAL8TimeSeries(
        lal_timeseries, float(1/sampling_frequency)
    )
    data = TimeSeries(
        lal_timeseries.data.data,
        epoch=lal_timeseries.epoch,
        dt=lal_timeseries.deltaT
    )

    # define some attributes in ifo
    ifo.strain_data.roll_off=roll_off
    ifo.sampling_frequency = sampling_frequency
    ifo.duration = duration
    ifo.maximum_frequency = maximum_frequency
    ifo.minimum_frequency = minimum_frequency

    ifo.strain_data.set_from_gwpy_timeseries(data)
    
    logger.info("Downloading psd data for ifo {}".format(det))
    
    # compute the psd
    if det in psds.keys():
        print("Using pre-computed psd from results file")
        ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psds[det][: ,0], psd_array=psds[det][:, 1]
        )
    else:
        # print('Error: PSD is missing!')
        # exit()
        psd_end_time = start_time
        psd_start_time = start_time - 32*duration
        psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time, sample_rate=4096)

        lal_timeseries = data.to_lal()
        lal.ResampleREAL8TimeSeries(
            lal_timeseries, float(1/sampling_frequency)
        )
        data = TimeSeries(
            lal_timeseries.data.data,
            epoch=lal_timeseries.epoch,
            dt=lal_timeseries.deltaT
        )

        psd_alpha = 2 * roll_off / duration                                         
        psd = psd_data.psd(                                                       
            fftlength=duration, overlap=0.5*duration, window=("tukey", psd_alpha), method="median"
        )

        ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
            frequency_array=psd.frequencies.value, psd_array=psd.value
        )

    ifo_list.append(ifo)


# define waveform generator
waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )
)

results = []

# setting the liklihood object and calculating the log likelihood.
#priors = bilby.core.prior.dict.PriorDict.from_json('GW150914_prior.json')

# Loading in the calibration envelope model.
# for ifo in ifo_list:
#     model = bilby.gw.calibration.Precomputed
#     det = ifo.name

#     ifo.calibration_model = model.from_envelope_file(
#     calib_file_paths[det],
#     frequency_array=ifo.frequency_array[ifo.frequency_mask],
#     n_nodes=10,
#     label=det,
#     n_curves=1000,
#     )

for ifo in ifo_list:
    ifo.calibration_model = bilby.gw.calibration.CubicSpline(f"recalib_{ifo.name}_",
                minimum_frequency=ifo.minimum_frequency,
                maximum_frequency=ifo.maximum_frequency,
                n_points=10)


# define likelihood object
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors = priors,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
    distance_marginalization=True,
    time_marginalization=True,
    distance_marginalization_lookup_table='TD.npz',
    calibration_marginalization=False,

)

# Use random parameters.
parameters = samples.iloc[3].to_dict()

reference_values = dict(luminosity_distance = priors['luminosity_distance'],
                        geocent_time = priors['geocent_time'])

# calculate log likelihood.
likelihood.parameters.update(parameters)
likelihood.parameters.update(reference_values)
ln_likelihood_value = likelihood.log_likelihood_ratio()
results.append(ln_likelihood_value)

print("log likelihood", ln_likelihood_value)    
GWOSC_value = parameters['log_likelihood']
print('difference = ', ln_likelihood_value - GWOSC_value)

Jitter time setting cannot be found. Use default setting.


15:09 bilby INFO    : Downloading analysis data for ifo H1
15:09 bilby INFO    : Downloading psd data for ifo H1
15:09 bilby INFO    : Downloading analysis data for ifo L1
15:09 bilby INFO    : Downloading psd data for ifo L1
15:09 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file
Using pre-computed psd from results file


15:09 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
/home/shunyin.cheung/.conda/envs/altenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
15:09 bilby INFO    : Building lookup table for distance marginalisation.
100%|██████████| 400/400 [00:43<00:00,  9.16it/s]


log likelihood 24.67531008663561
difference =  -0.041642678257016286
